In [ ]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("OPEN_API_KEY")

In [ ]:
import requests
import json

from langchain.tools import BaseTool
from langchain.agents import AgentType
from typing import Optional, Type
from pydantic import BaseModel, Field

class WeatherDataInput(BaseModel):
    """Get weather data input parameters."""
    limit: Optional[int] = Field(
        None, description="The max data count to return, return all by default")
    offset: Optional[int] = Field(
        None, description="The start index of data to return, start from 0 by default")
    format: str = Field(
        'json', description="The data format to return, json format by default")
    location_name: Optional[str] = Field(
        None, description="The cities in Taiwan, return all cities by default")
    element_name: Optional[list] = Field(
        None, description="The weather factors, return all by default")
    sort: Optional[str] = Field(
        None, description="Sort by 'startTime', 'endTime', no sort by default")
    start_time: Optional[list] = Field(
        None, description="The time factor, format as 'yyyy-MM-ddThh:mm:ss', return all by default")
    time_from: Optional[str] = Field(
        None, description="The start time for time range filter, format as 'yyyy-MM-ddThh:mm:ss', return all by default")
    time_to: Optional[str] = Field(
        None, description="The end time for time range filter, format as 'yyyy-MM-ddThh:mm:ss', return all by default")


class WeatherDataTool(BaseTool):
    name = "get_weather_data"
    description = "Get the weather data for Taiwan"

    def _run(self, input: WeatherDataInput):
        weather_data_results = get_weather_data(input)

        return weather_data_results

    def _arun(self, input: WeatherDataInput):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = WeatherDataInput


def get_weather_data(limit=None, offset=None, format='json', location_name=None,
                     element_name=None, sort=None, start_time=None, time_from=None, time_to=None):

    url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001'

    headers = {'accept': 'application/json'}

    query = {
        'Authorization': 'CWB-B32D114C-29B2-46D0-BF64-291C514502A0', 'format': format}

    if limit is not None:
        query['limit'] = limit

    if offset is not None:
        query['offset'] = offset

    if location_name is not None:
        query['locationName'] = ','.join(location_name)

    if element_name is not None:
        query['elementName'] = ','.join(element_name)

    if sort is not None:
        query['sort'] = sort

    if start_time is not None:
        query['startTime'] = ','.join(start_time)

    if time_from is not None:
        query['timeFrom'] = time_from

    if time_to is not None:
        query['timeTo'] = time_to

    response = requests.get(url, headers=headers, params=query)

    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code



In [ ]:
# Tools as OpenAI Functions
# Make sure langchain version after 0.0.202
# pip install --upgrade --force-reinstall langchain

import json
from langchain.tools import MoveFileTool, format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-3.5-turbo-0613")

# tools = [TravelPOITool(), TravelTicketTool(),
#         TravelExpTool(), WeatherDataTool()]
tools = [WeatherDataTool()]

# 取得 MoveFileTool 所有的 functions 接口
functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:

question = "幫我查台北市天氣"


In [ ]:
# tool_result = open_ai_agent.run(question)
ai_message = model.predict_messages(
    [HumanMessage(content=question)], functions=functions
)


In [ ]:
ai_message

In [ ]:
_args = json.loads(
    ai_message.additional_kwargs['function_call'].get('arguments'))
print(_args)
tools[0]
print(tools)

In [ ]:
tool_result = tools[0](_args)
print("Answer: ", tool_result)